In [ ]:
from pathlib import Path
import time
from random import Random

import IPython.display as display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation

from materiales import describir_imagen, descargar_sipi

In [ ]:
random = Random(42)  # Para reproducibilidad; cambiar para obtener distintos resultados

# Introducción al procesamiento de imágenes

## 1. Imágenes digitales y su representación

Una imagen digital es una representación bidimensional de una escena visual.
Existen dos tipos de representaciones digitales de imágenes: **raster** y
**vectorial**.

- **Raster**: Las imágenes raster son representadas como una matriz de píxeles.
  Cada píxel tiene un valor que representa el color de la imagen en esa
  posición. Las imágenes raster son las más comunes en aplicaciones de
  procesamiento de imágenes.

- **Vectorial**: Las imágenes vectoriales son representadas como una colección
  de objetos geométricos, como líneas, círculos y polígonos. Las imágenes
  vectoriales son comúnmente usadas en aplicaciones de diseño gráfico.

### 1.1 Los modelos de color

El modelo de color es un método para representar colores en una imagen.
Cuando haces zoom a un monitor de computadora, verás que la imagen está
compuesta por pequeños puntos de colores; a su vez, cada uno de estos puntos
está compuesto por tres diodos emisores de luz (LED) de color rojo, verde y
azul, que son los colores primarios de la luz (o *additivos*).
A este modelo de color se le conoce como **RGB** (por sus siglas en inglés:
*Red*, *Green* y *Blue*).

<figure>
    <img src="img/monitor.jpg"
         alt="Zoom a monitor de computadora.">
    <figcaption>
    <strong>Figura 1.</strong> Zoom al monitor de la computadora del autor.
    La imagen muestra la punta del cursor.
    </figcaption>
</figure>

En el modelo RGB, cada color es una tríada de números $(r, g, b)$, donde $r$,
$g$ y $b$ son los valores de intensidad de los colores rojo, verde y azul 
respectivamente.
Entre más alto sea el valor de un color, más brillante e intenso será en la
imagen.
Estos valores usualmente ocupan 1 byte cada uno, por lo que pueden tomar
valores entre 0 y 255.
Así, por ejemplo, el rojo puro se representa como $(255, 0, 0)$, el gris
medio como $(128, 128, 128)$ y el blanco como $(255, 255, 255)$.
En los sitios web, los colores se representan en hexadecimal con RGB, de manera
que ocupan 6 dígitos hexadecimales, dos para cada componente; por ejemplo, el
rojo `#FF0000`, el gris medio `#808080` y el blanco `#FFFFFF`.

A continuación mostramos un widget de selección de color que nos permite
explorar el modelo RGB y otros modelos más.

In [ ]:
# NOTA: Necesitas correr este notebook interactivamente para ver el widget.
widgets.ColorPicker()

Si haces clic en el las flechas a la derecha de "RGB", podrás seleccionar
el modelo *HSL*.
Este modelo representa los colores como una tríada de números $(h, s, l)$,
donde
- $h$ es un ángulo ($0^\circ$ a $360^\circ$) que representa el tono del
  color (los colores están organizados en un disco arcoíris),
- $s$ es la saturación o pureza del color (0% a 100%), y
- $l$ es la luminosidad (la cantidad de luz que refleja el color).

<figure>
    <img src="https://upload.wikimedia.org/wikipedia/commons/1/13/Color_solid_comparison_hsl_hsv_rgb_cone_sphere_cube_cylinder.png"
         alt="Modelos de color RGB, HSL y HSV."
         style="height: 400px;">
    <figcaption><strong>Figura 2.</strong>
    Modelos de color RGB, HSL y HSV. Imagen de <a href="https://commons.wikimedia.org/wiki/File:Color_solid_comparison_hsl_hsv_rgb_cone_sphere_cube_cylinder.png">SharkD</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a>, via Wikimedia Commons.
    </figcaption>
</figure>

### 1.2 Imágenes vectoriales

Dado que casi todo el procesamiento de imágenes se realiza sobre imágenes
raster, no se profundizará en las imágenes vectoriales.
Sin embargo, es importante mencionar que las imágenes vectoriales tienen
ventajas sobre las imágenes raster, como la escalabilidad y la facilidad para
realizar modificaciones.
En la web es común encontrar imágenes vectoriales en el formato estándar **SVG**
(*Scalable Vector Graphics*), basado en XML.

Por ejemplo, el siguiente código crea un pino con tres triángulos y un
rectángulo.
Nótese el uso de colores en el atributo `fill` y el uso de coordenadas
relativas al centro del lienzo en el atributo `viewBox`.

```xml
<svg width="200" height="300" viewBox="-100 -100 200 200">
  <polygon points="0,0 80,120 -80,120" fill="#234236" />
  <polygon points="0,-40 60,60 -60,60" fill="#0C5C4C" />
  <polygon points="0,-80 40,0 -40,0" fill="#38755B" />
  <rect x="-20" y="120" width="40" height="30" fill="brown" />
</svg>
```

<svg width="200" height="300" viewBox="-100 -100 200 200">
  <polygon points="0,0 80,120 -80,120" fill="#234236" />
  <polygon points="0,-40 60,60 -60,60" fill="#0C5C4C" />
  <polygon points="0,-80 40,0 -40,0" fill="#38755B" />
  <rect x="-20" y="120" width="40" height="30" fill="brown" />
</svg>

En [este enlace](https://youtu.be/kBT90nwUb_o) puedes ver un video tutorial
sobre cómo crear imágenes vectoriales en SVG usando el editor de código.
En la práctica, es más común usar un editor gráfico como
[Inkscape](https://inkscape.org/), con el que puedes crear imágenes vectoriales
[bastante complejas](https://youtu.be/1U4hVbvRr_g).
Estoy seguro de que encontrarás útil aprender a usar Inkscape, ya que es un
programa de código abierto que te permitirá ilustrar tus proyectos de
investigación, presentaciones y publicaciones.

### 1.3 Modelo Raster: La matriz de píxeles como imagen

Vamos a dibujar en un lienzo de $96 \times 96$ píxeles la gráfica de la
función
$$f(x, y) = (1 - x/2 + x^5 + y^3) \exp(-x^2 - y^2).$$
En los intervalos $x \in [-3, 3]$ e $y \in [-3, 3]$.

In [ ]:
x, y = np.linspace(-3, 3, 96), np.linspace(-3, 3, 96)
x, y = np.meshgrid(x, y)

# Calculamos la matriz z como función de x e y
z = (1 - x / 2 + x**5 + y**3) * np.exp(-(x**2) - y**2)

A continuación mostramos esta matriz usando `imshow` de `matplotlib`.

In [ ]:
fig, ax = plt.subplots()

# Usa plt.colormaps() para ver una lista de mapas de colores disponibles
# además de "twilight". Prueba con "magma", "viridis", etc.
im = ax.imshow(z, cmap="twilight", origin="lower", extent=[-3, 3, -3, 3])
fig.colorbar(im, ax=ax)

Esta gráfica de función ya tiene una representación como imagen digital, sin
embargo, las imágenes digitales suelen tener valores enteros entre 0 y 255.
En nuestro caso, la imagen tiene un único valor de intensidad para cada píxel,
por lo que es una imagen en escala de grises.
Para representarla visualmente la reescalaremos a valores enteros entre 0 y 255.

En general tenemos la siguiente regla de dedo para identificar el tipo de
imagen:
- Si la imagen es de tipo *bool*, es decir, solo tiene valores 0 o 1, entonces
  es una imagen binaria, también llamada *máscara* (como la cinta adhesiva que
  usamos para pintar sin manchar).
- Si la imagen tiene un solo canal de color, es una imagen en escala de grises.
- Si la imagen tiene tres canales de color, es una imagen en color RGB.
- Si la imagen tiene cuatro canales de color, es una imagen en color RGBA.
- Si la imagen es de tipo *float*, sus valores están en el rango $[0, 1]$,
  miebtras que si es de tipo *int*, sus valores están en el rango $[0, 255]$.

In [ ]:
# Normalizamos z
max_z, min_z = z.max(), z.min()
z_norm = (z - min_z) / (max_z - min_z)

describir_imagen(z_norm, "z_norm")

# Renormalizamos al rango [0, 255] en enteros (1 byte por pixel)
z_int = (z_norm * 255).astype(np.uint8)

fig, ax = plt.subplots()
ax.imshow(z_int, cmap="gray", origin="lower", extent=[-3, 3, -3, 3])
_ = ax.axis("off")

In [ ]:
# Obtener un mapa de colores
nombre_mapa = "twilight"
cmap = plt.get_cmap(nombre_mapa)

# Observamos unos cuantos valores de colores:
valores = [0, 0.25, 0.5, 0.75, 1]
colores = cmap(valores)

print(
    f"El mapa de colores '{nombre_mapa}' es de tipo "
    f"{type(cmap).__name__} y produce colores RGBA de 0 a 1."
)

for valor, color in zip(valores, colores):
    print(f"{nombre_mapa}({valor})\t=\t{color}")

In [ ]:
z_color = cmap(z_norm)
describir_imagen(z_color, "z_color")

fig, ax = plt.subplots()
_ = ax.imshow(z_color, origin="lower", extent=[-3, 3, -3, 3])
_ = ax.axis("off")

In [ ]:
# Renormalizar la matriz z para que sus valores estén entre 0 y 255 (enteros de 8 bits)
z_int = (z_color[..., :3] * 255 + 0.5).astype(np.uint8)

describir_imagen(z_int, "z_int")

Más adeltante te mostraré cómo guardar esta imagen.

#### Bases de datos de imágenes

En internet puedes encontrar muchas bases de datos de imágenes que puedes usar
para tus proyectos de investigación y educación.
Existen, sin embargo, algunas restricciones en el uso de estas imágenes.
Por ejemplo, algunas bases de datos requieren que se cite la fuente de las
imágenes, otras no permiten el uso comercial, y otras requieren que se
compartan los resultados de la investigación.

El Instituto de Procesamiento de Señales e Imágenes de la Universidad California
del Sur (**USC-SIPI**)  ha creado una base de datos de imágenes digitales con
el propósito de facilitar la investigación en el área.
Se puede acceder a la base de datos en el siguiente enlace:
https://sipi.usc.edu/database/

In [ ]:
# Descargamos la base de datos miscelánea de la USC-SIPI
descargar_sipi("misc")
sipi_dir = Path("sipi")

In [ ]:
archivos = list((sipi_dir /"misc").iterdir())
print(f"Hay {len(archivos)} imágenes en la base de datos miscelánea.")

In [ ]:
archivo_imagen = random.choice(archivos)
imagen = plt.imread(archivo_imagen)  # Aquí se lee la imagen
describir_imagen(imagen, archivo_imagen.name)

# Mostrar la imagen
fig, ax = plt.subplots()
if imagen.ndim == 3:
    img = ax.imshow(imagen)
else:
    img = ax.imshow(imagen, cmap="gray")
_ = ax.axis("off")

## 2. Uso de NumPy para el procesamiento de imágenes

Scikit-Image es una biblioteca de Python que provee una colección de algoritmos
para el procesamiento de imágenes.
Es parte del ecosistema de Scipy y es de código abierto.
Por el momento nos limitaremos a usar Scikit-Image para cargar y mostrar
imágenes, y usaremos NumPy para manipularlas.
En la siguiente sección veremos cómo usar Scikit-Image para ajustar las imágenes
con poderosos algoritmos de procesamiento de imágenes.

In [ ]:
import skimage as ski

In [ ]:
archivo_imagen = sipi_dir / "misc" / "5.2.08.tiff"

# Cargar y mostrar una imagen con Scikit-Image
img = ski.io.imread(archivo_imagen)
ski.io.imshow(img)

In [ ]:
# Para calcular el negativo de la imagen, restamos cada valor de 255
img = ski.io.imread(sipi_dir / "misc" / "5.2.08.tiff"
)
img_neg = 255 - img
ski.io.imshow(img_neg)

### Uso de índices de Numpy en la manipulación de imágenes

In [ ]:
img[140:180, 160:190] = 255  # Colocar un rectángulo blanco en la imagen
img[155:200, 280:315] = 255  # ... y otro más
ski.io.imshow(img)

In [ ]:
n_renglones = img.shape[0]
img[np.arange(n_renglones) % 64 == 0, :] = 0
ski.io.imshow(img)

In [ ]:
mascara = img > 128
describir_imagen(mascara, "mascara")
ski.io.imshow(mascara)

In [ ]:
img[img < 85] = 0
img[(img >= 85) & (img <= 170)] = 127
img[img > 170] = 255
ski.io.imshow(img)

### Uso básico de NumPy para manipular imágenes a color

In [ ]:
archivo_imagen = sipi_dir / "misc/4.2.03.tiff"
img = ski.io.imread(archivo_imagen)
describir_imagen(img)

fig, ax = plt.subplots()
ski.io.imshow(img)
_ = ax.axis("off")

In [ ]:
img = ski.data.astronaut()
# Descomponer la imagen en sus canales de color
imgs = (img.copy(), img.copy(), img.copy())
for i in range(3):
    complemento = [k for k in range(3) if k != i]
    imgs[i][:, :, complemento] = 0  # Apagar los otros dos canales

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0, 0].imshow(imgs[0])
ax[0, 0].set_title("Canal rojo")
ax[0, 1].imshow(imgs[1])
ax[0, 1].set_title("Canal verde")
ax[1, 0].imshow(imgs[2])
ax[1, 0].set_title("Canal azul")
ax[1, 1].imshow(imgs[0] + imgs[1] + imgs[2])
ax[1, 1].set_title("Imagen compuesta")
for i, j in np.ndindex(2, 2):
    ax[i, j].axis("off")

### Ajuste de la saturación de una imagen

El modelo HSV es más intuitivo que el modelo RGB, ya que el tono, la saturación
y la luminosidad son conceptos más fáciles de entender que la combinación de
rojo, verde y azul.

La siguiente imagen muestra un vestido que se hizo famoso en 2015 por la
diferencia de percepción del color entre las personas.
El vestido es blanco con dorado, pero debido a que la foto fue tomada a
contraluz, algunas personas perciben el vestido como azul y oscuro.

In [ ]:
import urllib.request
url = "https://upload.wikimedia.org/wikipedia/en/2/21/The_dress_blueblackwhitegold.jpg"
with urllib.request.urlopen(url) as response:
    img = plt.imread(response, format="jpeg")

describir_imagen(img, "El vestido")

aux = ski.color.rgb2hsv(img)  # Convertir la imagen a HSV

# Mostar la imagen y su histograma de color
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img)
ax[0].set_title("Imagen original")
ax[0].axis("off")
ax[1].hist(aux[:, :, 0].ravel(), bins=50, density=True)
ax[1].set_xlabel("Tono")
ax[1].set_ylabel("Densidad (%)")
ax[1].spines["top"].set_visible(False)
ax[1].spines["right"].set_visible(False)
ax[1].set_title("Histograma de color")
fig.tight_layout()

Vale la pena observar que los valores del arreglo de la imagen en formato HSV
están normalizados entre 0 y 1, incluyendo el valor del ángulo del tono.

In [ ]:
# Saturar la imagen
hsv_img = aux.copy()
hsv_img[:, :, 1] = 1.0
img_saturada = ski.color.hsv2rgb(hsv_img)

# Desaturar la imagen
hsv_img = aux.copy()
hsv_img[:, :, 1] = 0.0
img_desaturada = ski.color.hsv2rgb(hsv_img)

# Mostrar las imágenes
fig, ax = plt.subplots(1, 3)
ski.io.imshow(img, ax=ax[0])
ax[0].set_title("Imagen original")
ski.io.imshow(img_saturada, ax=ax[1])
ax[1].set_title("Imagen saturada")
ski.io.imshow(img_desaturada, ax=ax[2])
ax[2].set_title("Imagen desaturada")
for i in range(3):
    ax[i].axis("off")  # Ocultar los ejes
fig.tight_layout()  # Ajustar el espacio entre las figuras


Si ves que en tu grupo no hay consenso sobre el color del vestido, no te
preocupes, es normal, sólo recuerda que las discusiones sobre el color del
vestido no son tan importantes como las discusiones sobre el procesamiento de
imágenes.
Si te interesa, esta imagen ilustra la explicación de por qué ocurre este
fenómeno:

<figure>
    <img src="https://upload.wikimedia.org/wikipedia/commons/0/09/Wikipe-tan_wearing_The_Dress.svg"
         alt="Dos interpretaciones del color de <em>el vestido</em>."
            style="height: 400px;">
    <figcaption><strong>Figura 3.</strong> Dos interpretaciones del color de
    <em>el vestido</em>.
    <a href="https://commons.wikimedia.org/wiki/File:Wikipe-tan_wearing_The_Dress.svg">
    via Wikimedia Commons</a>,
    <a href="http://creativecommons.org/licenses/by-sa/3.0/">CC BY-SA 3.0</a>.
    </figcaption>
</figure>

### Ajuste del tono de una imagen

In [ ]:
# Scikit-Image también cuenta con imágenes de prueba como esta:
img = ski.data.chelsea()
ski.io.imshow(img)
_ = plt.gca().axis("off")

In [ ]:
# Seleccionar los puntos con mayor canal rojo
mascara = img[:, :, 0] > 160
ski.io.imshow(mascara)
_ = plt.gca().axis("off")

In [ ]:
# Cambiar el color de los puntos seleccionados a verde
img[mascara] = [0, 255, 0]
ski.io.imshow(img)
_ = plt.gca().axis("off")

A continuación veamos cómo cambiar el tono usando el modelo HSV.

In [ ]:
img = ski.data.chelsea()
img_hsv = ski.color.rgb2hsv(img)

# Para cambiar el tono de la imagen, sumamos 0.5 al canal H y tomamos el módulo 1
# para renormalizarlo a valores entre 0 y 1.
img_hsv[:, :, 0] = (img_hsv[:, :, 0] + 0.5) % 1.0
img_nueva = ski.color.hsv2rgb(img_hsv)
fig, ax = plt.subplots()
ski.io.imshow(img_nueva, ax=ax)
ax.axis("off")

## 3. Operaciones punto a punto

Las operaciones punto a punto son operaciones que se realizan sobre todos y cada
uno de los píxeles de una imagen de manera individual e independiente.

### 3.1 Brillo y contraste

Si el valor de color en cada píxel representa la intensidad de la luz, entonces
el brillo y el contraste de una imagen se pueden ajustar sumando y multiplicando
respectivamente los valores de color de cada píxel.

En concreto, si $A$ es la matriz de la imagen, $b$ es el brillo y $c$ es el
contraste, entonces el brillo y el contraste de la imagen se pueden ajustar
como sigue:
$$f_{\text{contraste}}(A, c) = c \cdot A \qquad f_{\text{brillo}}(A, b) = A + b$$

Aquí hemos usado la convención de Numpy de que las operaciones aritméticas se
realizan punto a punto.


In [ ]:
def mostrar_imagen_con_histograma(imagen, figsize=(13, 5)):
    fig, ax = plt.subplots(1, 2, figsize=figsize)
    ski.io.imshow(imagen, ax=ax[0])
    ax[0].axis("off")

    # Crear un histograma de la imagen
    ax[1].hist(imagen.ravel(), bins=256, density=True)
    ax[1].set_xlabel("Intensidad")
    ax[1].set_ylabel("Densidad")
    ax[1].spines["top"].set_visible(False)
    ax[1].spines["right"].set_visible(False)
    fig.tight_layout()

In [ ]:
imagen = ski.data.chelsea()/255


def ajustar_brillo_y_contrasete(brillo=0.0, contraste=1.0):
    imagen_ajustada = contraste * imagen + brillo
    imagen_ajustada[imagen_ajustada > 1] = 1
    imagen_ajustada[imagen_ajustada < 0] = 0
    out.clear_output()
    with out:
        mostrar_imagen_con_histograma(imagen_ajustada)
        plt.show()

out = widgets.Output()
display.display(out)
gui = widgets.interact_manual(
    ajustar_brillo_y_contrasete, brillo=(-1.0, 1.0, 0.05), contraste=(0.5, 2.0, 0.05)
)
gui.widget.children[0].description = "Brillo"
gui.widget.children[1].description = "Contraste"
gui.widget.children[2].description = "Actualizar"
ajustar_brillo_y_contrasete()

### 3.2 Corrección Gamma

Cada monitor tiene una curva de respuesta de luminancia que no es lineal; es
decir, el brillo de cada pixel no es directamente proporcional a los valores
RGB que se le envían.
Esta curva de respuesta depende del fabricante y modelo del monitor, así como de
la configuración del mismo.
La **corrección gamma** es una transformación de ley de potencia que se utiliza
para corregir la luminancia de una imagen; está definida por la siguiente
ecuación:

$$V_{\text{out}} = V_{\text{in}}^\gamma$$

donde $V_{\text{in}}$ es el valor de entrada de la imagen, $V_{\text{out}}$ es
el valor de salida y $\gamma$ es el factor de corrección gamma.
Esta transformación manda valores de $[0, 1]$ a $[0, 1]$ de manera monótona
para cualquier valor de $\gamma > 0$.

In [ ]:
x = np.linspace(0, 1, num=1000)
gamma = np.logspace(-5, 5, num=7, base=2)

fig, ax = plt.subplots()
for g in gamma:
    ax.plot(x, x**g, label=f"$\gamma = {g:.3g}$")

ax.legend()
ax.set_aspect(1.0)
ax.set_xlabel("$V_{\mathrm{in}}$")
ax.set_ylabel("$V_{\mathrm{out}}$")
ax.set_title("Curvas de transferencia de un ajuste gamma")

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

In [ ]:
img = ski.io.imread(sipi_dir / "misc/4.1.02.tiff")

img = img/255  # Normalizar la imagen a valores entre 0 y 1
img_corregida = img**0.5  # Aplica la corrección gamma

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ski.io.imshow(img, ax=ax[0])
ax[0].set_title("Imagen original")
ax[0].axis("off")

ski.io.imshow(img_corregida, ax=ax[1])
ax[1].set_title("Imagen corregida ($\gamma = 0.5$)")
ax[1].axis("off")

fig.tight_layout()

In [ ]:
gamma = np.logspace(-4, 4, num=101, base=2)  # Desde 2**-4 hasta 2**4

fig, ax = plt.subplots()
img_obj = ax.imshow(img)
ax.axis("off")
titulo = ax.set_title(f"Corrección gamma ($\gamma = {gamma[0]:.3f}$)", loc="left")
fig.tight_layout()
plt.close(fig)


def actualizar(i):
    img_corregida = img ** gamma[i]
    titulo.set_text(f"Corrección gamma ($\gamma = {gamma[i]:.3f}$)")
    img_obj.set_data(img_corregida)


anim = animation.FuncAnimation(fig, actualizar, frames=len(gamma), interval=1000 / 12)

inicio = time.perf_counter()
anim.save("correccion_gamma.mp4", fps=12, codec="h264")
print(f"La animación se guardó en {time.perf_counter() - inicio:.2f} segundos.")

display.Video("correccion_gamma.mp4")

### 3.3 Mezcla de imágenes

In [ ]:
img1 = ski.io.imread(sipi_dir / "misc/house.tiff")/255
img2 = ski.io.imread(sipi_dir / "misc/4.2.03.tiff")/255

print(f"Las imágenes tienen forma {img1.shape} y {img2.shape}.")

# Sumar las imágenes y renormalizar
img3 = img1 + img2
img3[img3 > 1.0] = 1.0


fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(img1)
ax[0].set_title("Imagen 1")
ax[1].imshow(img2)
ax[1].set_title("Imagen 2")
ax[2].imshow(img3)
ax[2].set_title("Suma")

for i in range(3):
    ax[i].axis("off")

fig.tight_layout()

In [ ]:
alpha = np.linspace(0, 1, num=100)

fig, ax = plt.subplots()
img_obj = ax.imshow(img1)
ax.axis("off")
fig.tight_layout()
plt.close(fig)

def fotograma(i):
    img3 = (1 - alpha[i]) * img1 + alpha[i] * img2  # Combinación convexa
    img_obj.set_data(img3)

anim = animation.FuncAnimation(fig, fotograma, frames=len(alpha), interval=1000/24)

inicio = time.perf_counter()
anim.save("transicion.mp4", codec="h264", fps=24)
print(f"La animación se guardó en {time.perf_counter() - inicio:.2f} segundos.")

display.Video("transicion.mp4")

In [ ]:
x = np.linspace(0, 1, num=400)
y = np.linspace(0, 1, num=400)
x, y = np.meshgrid(x, y)
z = x * y

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
img_obj = ax.plot_surface(x, y, z, cmap="magma", label="$x \cdot y$")
ax.legend()
_ = ax.set_xlabel("x")
_ = ax.set_ylabel("y")
_ = ax.set_zlabel("z")

In [ ]:
img3 = img1*img2

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(img1)
ax[0].set_title("Imagen 1")
ax[1].imshow(img2)
ax[1].set_title("Imagen 2")
ax[2].imshow(img3)
ax[2].set_title("Producto")

for i in range(3):
    ax[i].axis("off")

fig.tight_layout()


## 4. Ajuste de imágenes con Scikit-Image

Scikit-Image contiene varias rutinas para ajustar imágenes.

In [ ]:
def mostrar_imagen_gris_con_histograma(imagen, ejes, intervalos=256):
    imagen = ski.img_as_float(imagen)  # Normalizar la imagen a valores entre 0 y 1.
    ax_img, ax_hist = ejes
    ax_img.imshow(imagen, cmap="gray")
    ax_img.axis("off")
    ax_hist.hist(
        imagen.ravel(), bins=intervalos, histtype="stepfilled", color="black", density=True
    )
    ax_hist.set_xlabel("Intensidad")
    ax_hist.set_ylabel("Densidad")
    ax_hist.spines["top"].set_visible(False)
    ax_hist.spines["right"].set_visible(False)

    # Mostrar la distribución acumulada
    img_cdf, bins = ski.exposure.cumulative_distribution(imagen, intervalos)
    ax_cdf = ax_hist.twinx()
    ax_cdf.plot(bins, img_cdf, "r")
    ax_cdf.set_yticks([])  # Ocultar los valores del eje y para la distribución acum.
    ax_cdf.spines["top"].set_visible(False)
    ax_cdf.spines["right"].set_visible(False)

    return ax_img, ax_hist, ax_cdf

In [ ]:
img = ski.data.camera()
fig, ejes = plt.subplots(1, 2, figsize=(10, 5))
mostrar_imagen_gris_con_histograma(img, ejes)

### 4.1 Ajuste de brillo, contraste y gamma

Una manera común de ajustar automáticamente el contraste de una imagen
es reescalar sus valores de intensidad para que ocupen todo el rango de
valores posibles.

In [ ]:
p2, p98 = np.percentile(img, (2, 98))
img_reescalada = ski.exposure.rescale_intensity(img, in_range=(p2, p98))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
mostrar_imagen_gris_con_histograma(img_reescalada, ejes=ax)

La **ecualización del histograma** es una técnica que ajusta el contraste de
una imagen reescalando sus valores de intensidad para que ocupen todo el rango
de valores posibles.
Esencialmente, la ecualización del histograma redistribuye los valores de
intensidad de manera que la distribución acumulada de los valores de intensidad
sea lo más parecida a una función lineal.